# Voting using Various Models

In [1]:
# increase the width of the notebook
from IPython.display import display, HTML, Markdown

display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import VotingRegressor
import numpy as np
import joblib

In [3]:
best_lasso = joblib.load("Models/best_lasso.joblib")
best_ridge = joblib.load("Models/best_ridge.joblib")
best_poly = joblib.load("Models/best_poly.joblib")
best_tree = joblib.load("Models/best_tree.joblib")
best_rf = joblib.load("Models/best_rf.joblib")
best_et = joblib.load("Models/best_et.joblib")
best_ada_boost = joblib.load("Models/best_ada_boost.joblib")
best_gb = joblib.load("Models/best_gb.joblib")
best_stack_reg = joblib.load("Models/best_stack_reg.joblib")

In [4]:
preprocessor = joblib.load("preprocessor_unfitted.joblib")

In [5]:
voting = VotingRegressor(
    estimators=[
        ("lasso",     best_lasso),
        ("ridge",     best_ridge),
        ("poly",      best_poly),
        ("tree",      best_tree),
        ("rf",        best_rf),
        ("et",        best_et),
        ("ada",       best_ada_boost),
        ("gb",        best_gb),
        ("stack",     best_stack_reg),
    ],
    n_jobs=-1
)

In [6]:
voting_pipeline = Pipeline([
    ("voting", voting)
])

In [7]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [8]:
# Split features/target
X_train = train.drop('Score', axis=1)
y_train = train['Score']
X_test  = test.drop('Score', axis=1)
y_test  = test['Score']

In [9]:
# Fit the voting pipeline
voting_pipeline.fit(X_train, y_train)

Pipeline(steps=[('voting',
                 VotingRegressor(estimators=[('lasso',
                                              Pipeline(steps=[('prep',
                                                               ColumnTransformer(transformers=[('num',
                                                                                                Pipeline(steps=[('scaler',
                                                                                                                 StandardScaler())]),
                                                                                                ['WhiteElo',
                                                                                                 'EloDif']),
                                                                                               ('cat',
                                                                                                Pipeline(steps=[('onehot',
                                           

In [10]:
# Predict & evaluate
y_pred = voting_pipeline.predict(X_test)
print("VotingRegressor performance:")
print(f"  MSE : {mean_squared_error(y_test, y_pred):.4f}")
print(f"  MAE : {mean_absolute_error(y_test, y_pred):.4f}")
print(f"   R² : {r2_score(y_test, y_pred):.4f}")

VotingRegressor performance:
  MSE : 0.2265
  MAE : 0.4582
   R² : 0.0382


In [11]:
joblib.dump(voting_pipeline, "Models/voting_reg_pipeline.joblib")

['Models/voting_reg_pipeline.joblib']